In [4]:
import os
import pandas as pd
import numpy as np
from math import floor, log10

In [81]:
np.random.seed(123)



def get_scaled_variable(x):
    def get_scale(x):

        # Spec
        if x == 0.0:
            scale = 1
            return scale
        magnitude = int(floor(log10(abs(x))))
        if magnitude == 0:
            scale = 1 / 10
        else:

            scale = (10 ** magnitude) ** (-1) / 10
        return scale

    scale =  get_scale(x)
    rslt = scale * x
    return rslt

candidates = np.random.uniform(-123.0, 87.0, size=100)
for x in candidates:

    rslt = get_scaled_variable(x)
    
    assert 0.0 <= abs(rslt) <= 1.0
    print '{:15.4f}{:15.4f}'.format(x, rslt)# TODO: Keep sign? 

        23.2585         0.2326
       -62.9107        -0.6291
       -75.3612        -0.7536
        -7.2239        -0.0000
        28.0885         0.2809
       -34.1476        -0.3415
        82.9605         0.8296
        20.8142         0.2081
       -22.0043        -0.2200
       -40.6553        -0.4066
       -50.9326        -0.5093
        30.1004         0.3010
       -30.8998        -0.3090
      -110.4676        -0.1105
       -39.4107        -0.3941
        31.9790         0.3198
       -84.6767        -0.8468
       -86.1551        -0.8616
       -11.3742        -0.1137
       -11.3162        -0.1132
        10.2242         0.1022
        55.3807         0.5538
        29.1356         0.2914
         5.3149         0.0000
        28.7131         0.2871
       -55.1786        -0.5518
       -47.0244        -0.4702
       -75.0647        -0.7506
       -61.3201        -0.6132
         9.5050         0.0000
      -103.6580        -0.1037
       -31.9228        -0.3192
       -